This notebook is for writing the filtering and evaluation script. 
The best way to do this, is to first filter all bboxes and then save them for each frame in every scene. 
Then, we peform evaluation on it by loading the bboxes of a given frame and its gt_boxes and calculating the metrics for them

In [1]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(".."))

First the code to process each frame, each scene, and all scenes

In [2]:
import stats_filter
import pandas as pd
from prototype_utils import bboxes_df_to_numpy_corners
from config import CONFIG

def process_frame(input_frame_path: str, frame_id: str, scene_save_path: str):
    """
    Args: 
        input_frame_path: path to load the bboxes' dataframe for the given frame. 
        frame_id: id of the frame of interst
        scene_save_path: the path to save the processed frame. Frame will be saved as feather file at scene_save_path/frame_id.feather
                        ensure that this directory exists before calling this function
        
    """
    #filter bboxes
    bboxes_df = pd.read_feather(input_frame_path)
    bboxes_df['aspect_ratio'] = bboxes_df['box_width'] / bboxes_df['box_length']
    bboxes_df['area'] = bboxes_df['box_width'] * bboxes_df['box_length']
    _, normal_bboxes_df, _ = stats_filter.filter_by_aspect_ratio(bboxes_df, 'aspect_ratio', CONFIG)
    square_filter_df = stats_filter.filter_squares_by_area(normal_bboxes_df, 'aspect_ratio', 'area', CONFIG)
    rect_filter_df = stats_filter.filter_rects_by_area(normal_bboxes_df, 'aspect_ratio', 'area', CONFIG)

    combined_df = pd.concat([square_filter_df, rect_filter_df]).drop_duplicates()
    combined_df.reset_index(drop=True, inplace=True)
    #apply nms
    _, selected_idxes = stats_filter.apply_nms_on_pseudo_labels(bboxes_df_to_numpy_corners(combined_df), CONFIG['NMS_IOU_THRESHOLD'])
    nms_df = combined_df.iloc[selected_idxes]

    nms_df.reset_index(drop=True, inplace=True)
    if not os.path.exists(scene_save_path):
        raise ValueError(f"Path {scene_save_path} does not exist. Please create the directory before calling this function")
    nms_df.to_feather(os.path.join(scene_save_path, f"{frame_id}.feather"))
    
def process_scene(input_scene_path: str, scene_id: str, output_save_path: str):
    """
    Args: 
        scene_id: the id of the scene
        input_scene_path: the path to the scene  {os.listdir(input_scene_path) will return the frames}
        output_save_path: the base path to save each scene. Each frame will be saved as feather file at output_save_path/scene_id/frame_id.feather
    """
    scene_save_path = os.path.join(output_save_path, scene_id)
    os.makedirs(scene_save_path, exist_ok=True)
    try:
        for input_frame in os.listdir(input_scene_path):
            input_frame_path = os.path.join(input_scene_path, input_frame)
            frame_id = input_frame.split(".")[0]
            process_frame(input_frame_path, frame_id, scene_save_path)
            
    except Exception as e:
        print(f"Error processing scene {scene_id} : {e}")
        
        
def process_all_scenes(input_path: str, output_save_path: str):
    """
    Args: 
        input_path: the path to the scenes {os.listdir(input_path) will return the scenes}
    """
    os.makedirs(output_save_path, exist_ok=True)
    for scene in os.listdir(input_path):
        input_scene_path = os.path.join(input_path, scene)
        process_scene(input_scene_path, scene, output_save_path)
    

Now, we load the intput_data_path(bbox src) and define output file path, refine bboxes and save it

In [4]:
import os
from config import CONFIG
home = os.path.join(os.path.expanduser("~"), CONFIG['HOME_PATH'][CONFIG['OS']])


if CONFIG['ROI']:
    input_pl_path = os.path.join(home, *CONFIG['BBOX_FILE_PATHS']['ROI']) # pl = pseudo labels
    filtered_pl_output_path = os.path.join(home, *CONFIG['FILTERED_BBOX_FILE_PATHS']['ROI'])

else:
    input_pl_path = os.path.join(home, *CONFIG['BBOX_FILE_PATHS']['FULL_RANGE']) 
    filtered_pl_output_path = os.path.join(home, *CONFIG['FILTERED_BBOX_FILE_PATHS']['FULL_RANGE'])



In [5]:
process_all_scenes(input_pl_path, filtered_pl_output_path)

Now, we load bboxes, its corresponding gt_box and calculate metrics

In [6]:
from av2.datasets.sensor.av2_sensor_dataloader import AV2SensorDataLoader
from pathlib import Path
from av2.structures.sweep import Sweep
dataset_path = Path(os.path.join(home, "dataset", "av2", "train"))
av2 = AV2SensorDataLoader(data_dir=dataset_path, labels_dir=dataset_path)

In [28]:
from prototype_utils import filter_cuboids_by_roi, extract_face_corners
from eval_metrics import compute_matches, compute_ap2, reorder_by_iou
from typing import Dict

def calculate_metrics_frame(input_pl_frame_path: str, frame_id: str,scene_id: str, scene_save_path: str, av2:AV2SensorDataLoader,  config: Dict):
    try: 
        #load ground truth
        if config['ROI']: 
            cuboids = av2.get_labels_at_lidar_timestamp(scene_id, int(frame_id)).vertices_m
            filtered_cuboids = filter_cuboids_by_roi(cuboids, config)
            gt_corners = extract_face_corners(filtered_cuboids)
        else:
            cuboids = av2.get_labels_at_lidar_timestamp(scene_id, int(frame_id)).vertices_m
            gt_corners = extract_face_corners(cuboids)
        
        #load pseudo_labels
        pseudo_labels_df = pd.read_feather(input_pl_frame_path)
        pseudo_labels_corners = bboxes_df_to_numpy_corners(pseudo_labels_df)
        # print(f"scene_save_path: {scene_save_path}")
        
        #compute matches
        _ , pseudo_label_matches, overlaps = compute_matches(gt_corners, pseudo_labels_corners)
        
        # pseudo_label_matches = reorder_by_iou(pseudo_label_matches, overlaps)
        
        gt_length = len(gt_corners)
        num_of_predictions = len(pseudo_labels_corners)
        
        mAP, precisions, recalls, precision, recall =compute_ap2(pseudo_label_matches,gt_length,num_of_predictions)
        
        # print(f" type of mAP: {type(mAP)} \n type of precision: {type(precision)} \n type of recall: {type(recall)} \n type of precisions: {type(precisions)} \n type of recalls: {type(recalls)}")
        # print(f"precisions.shape: {precisions.shape} \nrecalls.shape: {recalls.shape}")
        results_df = pd.DataFrame({
            "mAP": [mAP],
            "precision": [precision],
            "recall": [recall],
            "precisions": [precisions],
            "recalls": [recalls],
            "pseudo_label_matches": [pseudo_label_matches],
            "num_of_predictions": [num_of_predictions],
            "gt_length": [gt_length],
        })
        
        if not os.path.exists(scene_save_path):
            raise ValueError(f"Path {scene_save_path} does not exist. Please create the directory before calling this function")
        
        frame_save_path = os.path.join(scene_save_path, f"{frame_id}.feather")
        results_df.to_feather(frame_save_path)    
    except Exception as e:
        print(f"shape of filtered_cuboids: {filtered_cuboids.shape} for scene_id: {scene_id} and frame_id: {frame_id}")
        print(f"Error: {e} caught because Frame: {frame_id} in scene: {scene_id} does not have enough ground truth labels in ROI") 
        
        
def calculate_metrics_scene(input_pl_path: str, scene_id: str, base_save_path: str, av2: AV2SensorDataLoader):
    scene_save_path = os.path.join(base_save_path, scene_id)
    os.makedirs(scene_save_path, exist_ok=True)
    # count = 0
    for frame in os.listdir(input_pl_path): # frame is timsstamp.feather
        # count += 1
        input_frame_path = os.path.join(input_pl_path, frame)
        frame_id = frame.split(".")[0]
        # print(f"processing frame {frame}")
        # print(f"loading pseudo_labels from {input_frame_path}")
        # print(f"scene_save_path: {scene_save_path}")
        calculate_metrics_frame(input_frame_path, frame_id, scene_id, scene_save_path, av2, CONFIG)
        # if count == 1:
        #     break
    

def calculate_metrics_all_per_frame(pseudo_labels_path: str, av2: AV2SensorDataLoader, base_save_path):
    """
    Calculate the metrics for all the scenes
    """
    os.makedirs(base_save_path, exist_ok=True)
    # count = 0 
    for scene_id in os.listdir(pseudo_labels_path):
        # count += 1
        input_scene_path = os.path.join(pseudo_labels_path, scene_id)
        # print(f"loading pseudo_labels from {input_scene_path} for scne_id: {scene_id}" )
        calculate_metrics_scene(input_scene_path, scene_id, base_save_path, av2)
        # if count == 1:
        #     break


def calculate_total_metrics(base_save_path: str):
    """
    Calculate the total metrics for all the scenes
    """
    mAPs = []
    precisions = []
    recalls = []
    for scene_id in os.listdir(base_save_path):
        scene_path = os.path.join(base_save_path, scene_id)
        for frame in os.listdir(scene_path):
            frame_path = os.path.join(scene_path, frame)
            results_df = pd.read_feather(frame_path)
            mAPs.append(results_df['mAP'][0])
            precisions.append(results_df['precision'][0])
            recalls.append(results_df['recall'][0])
    mAP = sum(mAPs)/len(mAPs)
    precision = sum(precisions)/len(precisions)
    recall = sum(recalls)/len(recalls)
    return mAP, precision, recall

Calculate metrics for all filtered and unfiltered pseudo-labels

In [29]:
if CONFIG['ROI']:
    metrics_normal_output_path = os.path.join(home, *CONFIG['METRICS_FILE_PATHS']['ROI']['NORMAL'])
    metrics_filtered_output_path = os.path.join(home, *CONFIG['METRICS_FILE_PATHS']['ROI']['FILTERED'])
else:
    metrics_normal_output_path = os.path.join(home, *CONFIG['METRICS_FILE_PATHS']['FULL_RANGE']['NORMAL'])
    metrics_filtered_output_path = os.path.join(home, *CONFIG['METRICS_FILE_PATHS']['FULL_RANGE']['FILTERED'])


In [30]:
calculate_metrics_all_per_frame(input_pl_path, av2, metrics_normal_output_path)

shape of filtered_cuboids: (0,) for scene_id: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd and frame_id: 315968341560625000
Error: too many indices for array: array is 1-dimensional, but 3 were indexed caught because Frame: 315968341560625000 in scene: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd does not have enough ground truth labels in ROI
shape of filtered_cuboids: (0,) for scene_id: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd and frame_id: 315968341660158000
Error: too many indices for array: array is 1-dimensional, but 3 were indexed caught because Frame: 315968341660158000 in scene: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd does not have enough ground truth labels in ROI
shape of filtered_cuboids: (0,) for scene_id: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd and frame_id: 315968341760354000
Error: too many indices for array: array is 1-dimensional, but 3 were indexed caught because Frame: 315968341760354000 in scene: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd does not have enough ground truth labels in ROI
shape of f

In [31]:
calculate_metrics_all_per_frame(filtered_pl_output_path, av2, metrics_filtered_output_path)

shape of filtered_cuboids: (0,) for scene_id: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd and frame_id: 315968341560625000
Error: too many indices for array: array is 1-dimensional, but 3 were indexed caught because Frame: 315968341560625000 in scene: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd does not have enough ground truth labels in ROI
shape of filtered_cuboids: (0,) for scene_id: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd and frame_id: 315968341660158000
Error: too many indices for array: array is 1-dimensional, but 3 were indexed caught because Frame: 315968341660158000 in scene: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd does not have enough ground truth labels in ROI
shape of filtered_cuboids: (0,) for scene_id: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd and frame_id: 315968341760354000
Error: too many indices for array: array is 1-dimensional, but 3 were indexed caught because Frame: 315968341760354000 in scene: ff52c01e-3d7b-32b1-b6a1-bcff3459ccdd does not have enough ground truth labels in ROI
shape of f

In [32]:
mAP, precision, recall = calculate_total_metrics(metrics_normal_output_path)
print(f"mAP: {mAP} \nprecision: {precision}  \nrecall: {recall}")

mAP: 0.0295080541906726 
precision: 0.052353910811642  
recall: 0.225127005381434


In [33]:
mAP_refined, precision_refined, recall_refined = calculate_total_metrics(metrics_filtered_output_path) 

print(f"mAP_refined: {mAP_refined} \nprecision_refined: {precision_refined} \nrecall_refined: {recall_refined}")

mAP_refined: 0.032271806002957315 
precision_refined: 0.07312734458333706 
recall_refined: 0.16503157281820421


In [34]:
((mAP_refined - mAP)/mAP)*100

9.366093048447519